In [115]:
import requests
from bs4 import BeautifulSoup as bs
import lxml
import json
import constant
import ast
import re

In [2]:
def get_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}
    r = requests.get(url, headers = headers)
    return r

In [3]:
 r = get_html('https://spb.kassir.ru/bilety-na-koncert?sort=1')

In [176]:
print(r)

<Response [200]>


In [4]:
html = bs(r.text, 'html.parser')
#soup = html.find_all('script', type="application/ld+json")
#s = soup[1]
#s = str(soup[1].string)
#t = json.loads(s)
#print(t)
event = []
soup = html.find_all('div', class_="col-xs-2")
s = soup[1]

In [6]:
html_event = s.find('div', class_='event').attrs['data-ec-item']
print(html_event)
print(type(html_event))

{"name":"Подарочный сертификат KASSIR.RU","id":212224,"price":500,"category":"Концерт","category_id":3000,"image":"https://cdn.kassir.ru/spb/poster/d0/d09b6e0d1dfa8611707b935eb277c733.jpg","venueName":"","position":0,"cat1":"Концерт","cat2":"","eventId":212224,"eventName":"Подарочный сертификат KASSIR.RU","minPrice":500,"maxPrice":15000,"date":"2020-01-01 00:00:00","avail":0,"placementId":1252}
<class 'str'>


In [171]:
event = []
id_parse = html_event['id']
# получем минимальную и максимальную стоимость события
min_cost = html_event['minPrice']
max_cost = html_event['maxPrice']
# формируем стоимость
if min_cost == max_cost:
    cost = str(max_cost) + ' рублей'
else:
    cost = 'от ' + str(min_cost) + ' до ' + str(max_cost) + ' рублей'
# получаем дату события
date = html_event['date']
# если событие не один день
if type(date) == dict:
    date_start = re_format_kudago(html_event['date']['start_min'])
    date_stop = re_format_kudago(html_event['date']['start_max'])
else:
    date_start = re_format_kudago(date)
    date_stop = None
type_event = "Концерт"
# вшитый в страницу сктипт с параметрами события
dictonary_event = json.loads(str(s.find('script', type="application/ld+json").string))
img = dictonary_event['image']
title = dictonary_event['name']
discounted = 0
address_place = dictonary_event['location']['address']
# убираем Санк-Петербург из адреса
if address_place.startswith('Санкт'):
    address_spb = address_place.split(', ')
    address_place = ', '.join(address_spb[1:])
address = dictonary_event['location']['name'] + ", " + address_place
metro = None
link = dictonary_event['url']
event.insert(0, (id_parse,type_event, img, title, date_start, date_stop, cost, discounted, address, metro, link))
print(event)

[(12257, 'Концерт', 'https://cdn.kassir.ru/kassir.ru/poster/7f/7f8e0525bf368e6f0bd8b1073be7ca4e.jpg', 'Кентервильское привидение', '28 марта 2021', '25 апреля 2021', 'от 600 до 800 рублей', 0, 'Театр Мастерская, ул. Народная, дом 1', None, 'https://spb.kassir.ru/koncert/teatr-masterskaya/kentervilskoe-prividenie_2021-03-28')]


In [155]:
def re_format_kudago(date):
    if len(date) > 10:
        # '2021-04-28 20:00:00'
        date =  date[0:10]
    list_date = date.split('-')
# удаление нуля в числе 
    number = int(list_date[2])
    re_format_date = str(number) + " " + constant.date_format_kudago[int(list_date[1])-1] + " " + list_date[0]
    return re_format_date

In [150]:
x = '2021-04-28 20:00:00'
y =  x[0:10]
print(y)

2021-04-28


In [127]:
x = '{"name":"Гречанинов. Демественная литургия","id":778220,"price":300,"category":"Театр","category_id":3009,"image":"https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg","venueName":"Государственная академическая Капелла","position":1,"cat1":"Театр","cat2":"","eventId":778220,"eventName":"Гречанинов. Демественная литургия","minPrice":300,"maxPrice":450,"date":"2021-03-31 19:00:00","avail":390,"placementId":107}'


print(x)

{"name":"Гречанинов. Демественная литургия","id":778220,"price":300,"category":"Театр","category_id":3009,"image":"https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg","venueName":"Государственная академическая Капелла","position":1,"cat1":"Театр","cat2":"","eventId":778220,"eventName":"Гречанинов. Демественная литургия","minPrice":300,"maxPrice":450,"date":"2021-03-31 19:00:00","avail":390,"placementId":107}


In [128]:
w = re.sub(r'[^:]""',' ', x).replace(' "', ' ').replace('" ', ' ')
print(w)
y = x.replace('""', '"').replace(' "', ' ').replace('" ', ' ')


{"name":"Гречанинов. Демественная литургия","id":778220,"price":300,"category":"Театр","category_id":3009,"image":"https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg","venueName":"Государственная академическая Капелла","position":1,"cat1":"Театр","cat2":"","eventId":778220,"eventName":"Гречанинов. Демественная литургия","minPrice":300,"maxPrice":450,"date":"2021-03-31 19:00:00","avail":390,"placementId":107}


In [129]:
json.loads(w)

{'name': 'Гречанинов. Демественная литургия',
 'id': 778220,
 'price': 300,
 'category': 'Театр',
 'category_id': 3009,
 'image': 'https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg',
 'venueName': 'Государственная академическая Капелла',
 'position': 1,
 'cat1': 'Театр',
 'cat2': '',
 'eventId': 778220,
 'eventName': 'Гречанинов. Демественная литургия',
 'minPrice': 300,
 'maxPrice': 450,
 'date': '2021-03-31 19:00:00',
 'avail': 390,
 'placementId': 107}